In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MeasureControl
import shapely
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, MultiPoint, MultiLineString
from shapely.ops import split, snap
from shapely.affinity import scale
import matplotlib.pyplot as plt
import math

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

module_path = os.path.abspath(os.path.join(r'..\BTools\beratools\tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from BTools.beratools.tools.line_footprint_fixed import *


from label_centerlines import get_centerline

lcpath = gpd.read_file(r"C:\BERATools\Surmont_New_AOI\test_selected_lines\selected_lines-debug.shp")
cl = gpd.read_file(r"C:\BERATools\Surmont_New_AOI\test_selected_lines\centerlines.shp")
poly = gpd.read_file(r"C:\BERATools\Surmont_New_AOI\test_selected_lines\centerlines_corridor_poly.shp")

mapa = folium.Map([56.224917, -110.713430],
    zoom_start=17,
    tiles='cartodbpositron',
    max_zoom=24)

geoms = gpd.GeoDataFrame(geometry=[lcpath.geometry.iloc[0], cl.geometry.iloc[0], poly.geometry.iloc[0]], crs='EPSG:26912')
geoms['color'] = ['#E0DB4D', '#288519', '#553a70']
gjson = geoms.to_crs(epsg='4326').to_json()
# geoms.plot(color=geoms['color'])

folium.GeoJson(
    gjson,
    style_function=lambda feature: {
        "fillColor": feature["properties"]["color"],
        "color": feature["properties"]["color"],
        "weight": 2,
        "dashArray": "0, 0",
    },
).add_to(mapa)

In [ ]:
lcp = lcpath.geometry.iloc[0]
if centerline_is_valid(cl.geometry.iloc[0], lcp):
    print('Centerline is invalid')

center_line = regenerate_centerline(poly.geometry.iloc[0], lcp)

In [ ]:
type(center_line)

In [ ]:
geoms_cl = gpd.GeoDataFrame(geometry=[center_line], crs='EPSG:26912')
geoms_cl['color'] = ['#5E31DE']
gjson_cl = geoms_cl.to_crs(epsg='4326').to_json()

folium.GeoJson(
    gjson_cl,
    style_function=lambda feature: {
        "fillColor": feature["properties"]["color"],
        "color": feature["properties"]["color"],
        "weight": 2,
        "dashArray": "0, 0",
    },
).add_to(mapa)

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs(26912, 4326, always_xy=True)
start = shapely.force_2d(transform(transformer.transform, Point(list(center_line.geoms[0].coords)[0])))
end = shapely.force_2d(transform(transformer.transform, Point(list(center_line.geoms[1].coords)[-1])))

In [ ]:
folium.Marker(location=[start.y, start.x], popup='center').add_to(mapa)
folium.Marker(location=[end.y, end.x], popup='center').add_to(mapa)

In [ ]:
mapa.add_child(MeasureControl())
mapa

In [ ]:
mapa.save(r'D:\Temp\test.html')